In [1]:
import time
import numpy as np
import torch
import torchaudio

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import groupby
from operator import itemgetter
from matplotlib.collections import LineCollection

device = 'cuda'

In [2]:
from speech_parser import Audio
from speech_parser import SileroVAD
from speech_parser import OnlineSpeakerClustering, LongformOnlineClustering, OverclusteringWindow, OverclusteringWindowArray
from speech_parser import MSDD

/home/raid/miniconda3/envs/nemo/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
from utils import load_audio

In [4]:
# ocw = OverClusteringWindow(
#     torch.rand((200, 5, 192)),
#     200,
#     10
# )

In [5]:
ocwa = OverclusteringWindowArray(
    200,
    10
)

In [6]:
ocwa.extend(torch.rand((190, 5, 192)))

In [10]:
ocwa.extend(torch.rand((190, 5, 192)))

In [11]:
ocwa.cluster()

In [12]:
ocwa.grouped_embeds

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([[[0.4373, 0.5625, 0.4244,  ..., 0.5105, 0.4075, 0.5176],
          [0.5439, 0.4519, 0.4497,  ..., 0.6418, 0.6001, 0.4598],
          [0.3894, 0.4823, 0.6746,  ..., 0.5124, 0.6527, 0.4943],
          [0.4959, 0.4621, 0.5806,  ..., 0.2801, 0.5456, 0.5923],
          [0.3999, 0.5344, 0.5104,  ..., 0.3273, 0.5813, 0.5128]],
 
         [[0.4311, 0.5054, 0.5438,  ..., 0.4864, 0.5182, 0.5056],
          [0.4996, 0.5318, 0.5446,  ..., 0.5316, 0.5031, 0.4916],
          [0.5266, 0.4742, 0.4993,  ..., 0.5150, 0.5412, 0.4782],
          [0.4836, 0.4938, 0.4927,  ..., 0.5085, 0.5613, 0.4875],
          [0.4735, 0.4863, 0.5251,  ..., 0.5326, 0.5031, 0.4610]],
 
         [[0.2279, 0.3383, 0.4647,  ..., 0.5850, 0.2709, 0.1860],
          [0.3699, 0.5175, 0.2676,  ..., 0.2744, 0.4843, 0.5058],
          [0.2739, 0.3667, 0.3566,  ..., 0.4996, 0.4259, 0.6970],
          [0.1517, 0.5809, 0.5900,  ..., 0.7510, 0.5007, 0.2700],

In [5]:
ocw.cluster()

tensor([1, 4, 2, 1, 7, 1, 1, 2, 0, 1, 1, 8, 1, 1, 9, 1, 4, 1, 1, 1, 2, 1, 6, 1,
        2, 1, 7, 2, 1, 1, 6, 2, 2, 6, 2, 7, 1, 5, 1, 2, 1, 1, 2, 1, 0, 6, 4, 1,
        0, 2, 1, 1, 1, 2, 1, 2, 4, 2, 1, 0, 3, 1, 1, 9, 0, 1, 1, 6, 1, 2, 1, 1,
        6, 8, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 1, 1, 9, 2, 9, 1, 6,
        1, 1, 1, 1, 2, 6, 1, 1, 6, 1, 2, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 2, 6, 1,
        4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 7, 4, 8, 1, 1, 5, 2, 6, 1, 1, 1, 0, 1,
        2, 2, 2, 2, 2, 5, 1, 2, 9, 1, 1, 5, 2, 1, 2, 1, 0, 5, 2, 1, 2, 2, 9, 2,
        1, 1, 3, 1, 1, 2, 0, 1, 2, 0, 7, 1, 1, 2, 0, 9, 2, 2, 3, 1, 1, 8, 1, 1,
        1, 9, 1, 1, 1, 1, 0, 5])

In [6]:
ocw.labels

tensor([1, 4, 2, 1, 7, 1, 1, 2, 0, 1, 1, 8, 1, 1, 9, 1, 4, 1, 1, 1, 2, 1, 6, 1,
        2, 1, 7, 2, 1, 1, 6, 2, 2, 6, 2, 7, 1, 5, 1, 2, 1, 1, 2, 1, 0, 6, 4, 1,
        0, 2, 1, 1, 1, 2, 1, 2, 4, 2, 1, 0, 3, 1, 1, 9, 0, 1, 1, 6, 1, 2, 1, 1,
        6, 8, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 1, 1, 9, 2, 9, 1, 6,
        1, 1, 1, 1, 2, 6, 1, 1, 6, 1, 2, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 2, 6, 1,
        4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 7, 4, 8, 1, 1, 5, 2, 6, 1, 1, 1, 0, 1,
        2, 2, 2, 2, 2, 5, 1, 2, 9, 1, 1, 5, 2, 1, 2, 1, 0, 5, 2, 1, 2, 2, 9, 2,
        1, 1, 3, 1, 1, 2, 0, 1, 2, 0, 7, 1, 1, 2, 0, 9, 2, 2, 3, 1, 1, 8, 1, 1,
        1, 9, 1, 1, 1, 1, 0, 5])

In [7]:
ocw.get_labels(
    {
        0: 4,
        2: 3,
        4: 2,
        3: 0
    }
)

tensor([0, 2, 3, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0,
        3, 0, 0, 3, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 4, 0, 2, 0,
        4, 3, 0, 0, 0, 3, 0, 3, 2, 3, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
        0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 4, 0, 3, 4, 0, 0, 0, 3, 0, 0,
        2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 0,
        3, 3, 3, 3, 3, 0, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 4, 0, 3, 0, 3, 3, 0, 3,
        0, 0, 0, 0, 0, 3, 4, 0, 3, 4, 0, 0, 0, 3, 4, 0, 3, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 4, 0])

In [9]:
a = ocw.get_grouped_embeddings(
    {
        0: 4,
        2: 3,
        4: 2,
        3: 0
    }
)

In [11]:
for key, val in a.items():
    print(key)
    print(val.shape)

tensor(0)
torch.Size([5, 192])
tensor(2)
torch.Size([5, 192])
tensor(3)
torch.Size([5, 192])
tensor(4)
torch.Size([5, 192])


In [31]:
msdd = MSDD(
    threshold=0.8
)
titanet_l = msdd.speech_embedding_model
vad = SileroVAD(
    threshold=0.5
)
osc = OnlineSpeakerClustering()

scales = [1.5, 1.25, 1.0, 0.75, 0.5]
hops = [scale/4 for scale in scales]
a = Audio(
    scales, 
    hops, 
    speech_embedding_model=titanet_l,
    voice_activity_detection_model=vad,
    multi_scale_diarization_model=msdd,
    speaker_clustering=osc
)

[NeMo I 2025-02-21 16:01:18 cloud:58] Found existing object /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2025-02-21 16:01:18 cloud:64] Re-using file from: /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2025-02-21 16:01:18 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-02-21 16:01:19 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2025-02-21 16:01:19 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2025-02-21 16:01:19 modelPT:189] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2025-02-21 16:01:19 features:305] PADDING: 16
[NeMo I 2025-02-21 16:01:19 features:305] PADDING: 16
[NeMo I 2025-02-21 16:01:20 save_restore_connector:275] Model EncDecDiarLabelModel was successfully restored from /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.


In [32]:
# waveform, sr = load_audio('test.wav')
waveform, sr = load_audio('toefl_eg.mp3')
waveform = waveform[0]

In [33]:
proba, labels = a(waveform[:100_000])

In [34]:
proba.shape

torch.Size([0])

In [35]:
proba, labels = a(waveform[100_000:200_000])

In [9]:
proba

tensor([])

In [38]:
proba, labels = a(waveform[200_000:300_000])

In [10]:
proba, labels = a(waveform[200_000:1_000_000])

In [11]:
proba, labels = a(waveform[1_000_000:4_000_000])

In [12]:
# a(waveform[4_000_000:5_000_000])

In [39]:
merged = a.get_merged_speaker_segments()

In [40]:
merged

[SpeakerSegment(start=0, end=17.625, data=tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0015, -0.0009, -0.0007]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None)]

In [14]:
a.max_silence_per_segment_pct = 0.5

In [15]:
a.clear_merged_segments()

In [16]:
[s.duration for s in merged]

[17.625,
 32.5,
 11.0,
 1.0,
 72.5,
 4.75,
 4.625,
 11.5,
 3.625,
 10.5,
 10.75,
 10.25,
 4.5,
 1.0,
 6.375,
 19.75,
 5.875,
 10.25,
 6.25]

In [17]:
[(s.start, s.end, s.mask) for s in a.base_scale_segments[40:60]]

[(5.0, 5.5, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (5.125, 5.625, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (5.25, 5.75, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (5.375, 5.875, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (5.5, 6.0, tensor([1., 1., 1.,  ..., 0., 0., 0.])),
 (5.625, 6.125, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (6.125, 6.625, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (6.25, 6.75, tensor([0., 0., 0.,  ..., 1., 1., 1.])),
 (6.375, 6.875, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (6.5, 7.0, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (6.625, 7.125, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (6.75, 7.25, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (6.875, 7.375, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (7.0, 7.5, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (7.125, 7.625, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (7.25, 7.75, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (7.375, 7.875, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (7.5, 8.0, tensor([1., 1., 1.,  ..., 1., 1., 1.])),
 (

In [18]:
a.waveform

tensor([0.0000, 0.0000, 0.0000,  ..., 0.3207, 0.2932, 0.2494])

In [19]:
a.base_scale_segments[44].has_speech

True

In [20]:
merged

[SpeakerSegment(start=0, end=17.625, data=tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0015, -0.0009, -0.0007]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None),
 SpeakerSegment(start=17.875, end=50.375, data=tensor([-0.0037, -0.0012,  0.0005,  ...,  0.0037,  0.0032,  0.0023]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None),
 SpeakerSegment(start=50.875, end=61.875, data=tensor([ 1.6737e-03,  3.6514e-04, -5.9805e-05,  ..., -1.1843e-03,
         -1.0885e-03, -7.7725e-04]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None),
 SpeakerSegment(start=77.0, end=78.0, data=tensor([-6.1943e-02, -6.3724e-02, -6.5221e-02,  ...,  3.0402e-04,
          1.6657e-05,  2.6669e-04]), mask=tensor([1., 1., 1.,  ..., 0., 0., 0.]), speaker=1, transcription=None),
 SpeakerSegment(start=62.375, end=134.875, data=tensor([-0.0970, -0.0802, -0.1046,  ...,  0.0000,  0.0000,  0.0000]), mask=tensor([1., 1., 1.,  ..., 0., 0., 0.]), spea

In [30]:
i = 6
torchaudio.save('segmented.wav', merged[i].data.unsqueeze(0).cpu(), 16_000)
merged[i].speaker

2

In [26]:
merged[i].duration

11.0

In [56]:
a.base_scale_segments[1].mask.sum()

tensor(7168., device='cuda:0')

In [37]:
from speech_parser.audio import SpeakerSegment

In [59]:
sr = 16_000

def merge_segments(segments, speaker, sampling_rate=16000):
    
    first_segment = segments[0]
    start = first_segment.start
    end = first_segment.start
    normal_segment_duration = first_segment.end - first_segment.start
    waveforms = []
    
    for segment in segments:
        increment = segment.end - end
        assert increment <= normal_segment_duration, f"Issue in the matching section of get_merged_speaker_segments(), {increment} > {normal_segment_duration}"

        concat_length = int(increment*sampling_rate)
        waveforms.append(segment.data[-concat_length:])
        
        end = segment.end
        
    waveform = torch.cat(waveforms)
    return SpeakerSegment(start, end, waveform, speaker)
    # print([segment.start for segment in segments])
    # pass

In [60]:
segment.data.shape

torch.Size([8000])

In [61]:
active_speakers  = {}

merged_segments = []

for segment in a.base_scale_segments:
    curr_active_speakers = list(active_speakers.items())
    for speaker, active_segments in curr_active_speakers:
        # end the speaker segment if the start of the new segment is past the end of the old
        if (speaker in segment.speakers) and (segment.start <= active_segments[-1].end):
            active_speakers[speaker].append(segment)
        else:
            # merge and end speakers
            merged_segments.append(merge_segments(active_segments, speaker))
            active_speakers.pop(speaker)
        
    for speaker in segment.speakers:
        if speaker not in active_speakers:
            # start speaker
            active_speakers[speaker] = [segment]

In [62]:
merged_segments

[SpeakerSegment(start=62.5, end=120.75, data=tensor([-0.0164, -0.0267,  0.0036,  ..., -0.0011, -0.0012, -0.0008],
        device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=120.75, end=134.75, data=tensor([-0.0006, -0.0006, -0.0004,  ...,  0.0000,  0.0000,  0.0000],
        device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=136.25, end=138.5, data=tensor([ 1.0500e-04,  3.5520e-05, -9.4904e-07,  ..., -1.4732e-02,
         -1.2964e-02, -1.2140e-02], device='cuda:0'), speaker=0, transcription=None),
 SpeakerSegment(start=136.25, end=141.0, data=tensor([ 1.0500e-04,  3.5520e-05, -9.4904e-07,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00], device='cuda:0'), speaker=1, transcription=None),
 SpeakerSegment(start=141.25, end=145.75, data=tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -7.6305e-05,
         -5.3419e-05, -3.3926e-05], device='cuda:0'), speaker=1, transcription=None),
 SpeakerSegment(start=146.0, end=156.0, data=tensor([0., 0.

In [66]:
merged_segments[-2].data

tensor(0.4973, device='cuda:0')

In [30]:
active_speakers

{}

In [19]:
def process_timeline(data):
    # Convert None to empty list for consistency
    timeline = [[] if x is None else sorted(x) for x in data]
    return timeline

def merge_segments(timeline):
    merged_segments = []
    
    # Find continuous segments for each speaker
    for speaker in set([spk for t in timeline for spk in t]):
        start_idx = None
        
        for t, speakers in enumerate(timeline):
            if speaker in speakers:
                if start_idx is None:
                    start_idx = t
            elif start_idx is not None:
                # Add segment
                merged_segments.append({
                    'speaker': speaker,
                    'start': start_idx * 0.25,  # Convert to seconds
                    'end': t * 0.25,            # Convert to seconds
                    'duration': (t - start_idx) * 0.25
                })
                start_idx = None
        
        # Handle segment that goes until the end
        if start_idx is not None:
            merged_segments.append({
                'speaker': speaker,
                'start': start_idx * 0.25,
                'end': len(timeline) * 0.25,
                'duration': (len(timeline) - start_idx) * 0.25
            })
    
    return sorted(merged_segments, key=lambda x: (x['start'], x['speaker']))

def generate_rttm(segments):
    rttm_lines = []
    
    for seg in segments:
        rttm_line = f"SPEAKER unknown 1 {seg['start']:.3f} {seg['duration']:.3f} <NA> <NA> SPEAKER_{seg['speaker']} <NA> <NA>"
        rttm_lines.append(rttm_line)
    
    return "\n".join(rttm_lines)

def create_visualization(segments, total_duration):
    plt.figure(figsize=(20, 5))
    
    # Get unique speakers and assign them y-coordinates
    unique_speakers = sorted(set(seg['speaker'] for seg in segments))
    speaker_to_y = {speaker: i for i, speaker in enumerate(unique_speakers)}
    
    # Create line segments for each speaker
    for speaker in unique_speakers:
        speaker_segments = [seg for seg in segments if seg['speaker'] == speaker]
        
        for seg in speaker_segments:
            plt.hlines(
                y=speaker_to_y[seg['speaker']],
                xmin=seg['start'],
                xmax=seg['end'],
                linewidth=4,
                label=f"Speaker {speaker}"
            )
    
    # Set axis limits to show the full timeline
    plt.xlim(0, total_duration)
    plt.ylim(-0.5, len(unique_speakers) - 0.5)
    
    # Set y-axis labels
    plt.yticks(
        range(len(unique_speakers)),
        [f"Speaker {speaker}" for speaker in unique_speakers]
    )
    
    # Remove duplicate labels in legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Customize the plot
    plt.xlabel("Time (seconds)")
    plt.ylabel("Speakers")
    plt.title("Speaker Diarization Timeline")
    plt.grid(True, alpha=0.3)
    
    # Add some padding to the plot
    plt.margins(x=0.02)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Save the plot
    plt.savefig('speaker_diarization.png', bbox_inches='tight')
    plt.close()

# Use your existing code to process the data and generate segments
timeline = process_timeline(timeline)
merged_segments = merge_segments(timeline)

# Calculate total duration in seconds
total_duration = len(timeline) * 0.25

# Generate RTTM
rttm_content = generate_rttm(merged_segments)
with open('output.rttm', 'w') as f:
    f.write(rttm_content)

# Create visualization with total duration
create_visualization(merged_segments, total_duration)

# Print segments for verification
for seg in merged_segments:
    print(f"Speaker {seg['speaker']}: {seg['start']:.2f}s - {seg['end']:.2f}s (duration: {seg['duration']:.2f}s)")

Speaker 0: 62.00s - 120.00s (duration: 58.00s)
Speaker 0: 120.25s - 134.00s (duration: 13.75s)
Speaker 0: 135.75s - 137.75s (duration: 2.00s)
Speaker 1: 135.75s - 140.25s (duration: 4.50s)
Speaker 1: 140.75s - 145.00s (duration: 4.25s)
Speaker 2: 145.50s - 155.25s (duration: 9.75s)
Speaker 2: 155.50s - 156.50s (duration: 1.00s)
Speaker 1: 156.75s - 160.00s (duration: 3.25s)
Speaker 2: 159.00s - 168.75s (duration: 9.75s)
Speaker 1: 168.50s - 178.25s (duration: 9.75s)
Speaker 2: 171.75s - 172.75s (duration: 1.00s)
Speaker 2: 175.25s - 175.75s (duration: 0.50s)
Speaker 2: 178.00s - 181.75s (duration: 3.75s)
Speaker 2: 182.00s - 187.25s (duration: 5.25s)
Speaker 1: 187.50s - 191.25s (duration: 3.75s)
Speaker 2: 191.00s - 191.25s (duration: 0.25s)
Speaker 1: 191.50s - 191.75s (duration: 0.25s)
Speaker 2: 191.50s - 197.00s (duration: 5.50s)
Speaker 1: 196.50s - 206.75s (duration: 10.25s)
Speaker 1: 207.00s - 215.75s (duration: 8.75s)
Speaker 2: 215.25s - 220.25s (duration: 5.00s)
Speaker 1: 